## Convert experiment results to excel files

In [1]:
from dbconfig import ensemble_selection_exp_results, ensemble_selection_results
import csv
import numpy as np
import os
from config import get_noniid_label_number_split_name
from collections import Counter

In [355]:
batch = 14
test_results = ensemble_selection_exp_results.find({"batch": batch})  # Find all experiment results
data = list(test_results)
length = len(data)
length

6421

In [356]:
for i in range(len(data)):
    del data[i]["reports"]
    del data[i]["config"]["preset_config"]
    del data[i]["config"]["indexes"]

In [357]:
output_data = []
headers = ['_id', 'batch','timestamp', 'method','average_accuracy', 'average_fscore', 'repeat_times',
           'partition', 'split', 'dataset', 'party_num', 'K','cluster_method']
data_length = len(data)
for i in range(data_length):
#     if data[i]["method"] == "oracle":
#         continue
    output_data.append([])
    for j in range(len(headers)):
        if j<=6:
            if j == 1:
                try:
                    batch_number = data[i]["batch"]
                except:
                    batch_number = 1
                output_data[-1].append(batch_number)
            else:
                output_data[-1].append(data[i][headers[j]])
        elif j<=11:
            if j == 10:
                output_data[-1].append("N"+str(data[i]["config"][headers[j]]))
            else:
                output_data[-1].append(data[i]["config"][headers[j]])
        else:
            try:
                cluster_method = data[i]["config"]["cluster_method"]
            except:
                cluster_method = data[i]["method"]
            output_data[-1].append(cluster_method)
            
#     for key in data[i]:
#         if key == "config":
#             for key_sub in data[i]["config"]:
#                 if i == 0:
#                     headers.append(key_sub)
#                 output_data[i].append(data[i]["config"][key_sub])
#         else:
#             if i == 0:
#                 headers.append(key)
#             output_data[i].append(data[i][key])
print(headers)
print(output_data[0])
print(output_data[1])
output_file = "exp_results_%d.csv" % batch
with open(output_file, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)
    for i in range(len(output_data)):
        writer.writerow(output_data[i])

['_id', 'batch', 'timestamp', 'method', 'average_accuracy', 'average_fscore', 'repeat_times', 'partition', 'split', 'dataset', 'party_num', 'K', 'cluster_method']
[ObjectId('633463a0994a01543706fd11'), 14, 1664377760, 'hierarchical_clustering_selection', 0.8652452672711408, 0.8630640195634217, 1, 'iid-diff-quantity', 'balanced', 'emnist', 'N100', 10, 'single']
[ObjectId('633463a9994a01543706fd13'), 14, 1664377769, 'hierarchical_clustering_selection', 0.8652452672711408, 0.8630640195634217, 1, 'iid-diff-quantity', 'balanced', 'emnist', 'N100', 10, 'centroid']


In [358]:
## Check Integrity

### Check Model Files

In [409]:
model_dirs = os.listdir("models/")
device = "cuda:1"
config_emnist_SpinalNet = [2, "SpinalNet", "cifar10", device, 1, 62, [100,200,400],["digits","letters","balanced"]]
config_cifar10_densenet = [12, "densenet", "cifar10", device, 3, 10, [50,100,200],['cifar10']]
# config_cifar10_effenetv2_l = [11, "effenetv2_l", "cifar10", device, 3, 10, [50,100,200],['cifar10']]
config_cifar10_SpinalNet = [10, "SpinalNet", "cifar10", device, 3, 10, [50,100,200],['cifar10']]
# config_cifar100_efficientnet_b7 = [22, "effenetv2_l", "cifar100", device, 3, 100, [5,10,20],['cifar100']]
# config_cifar100_SpinalNet = [23, "SpinalNet", "cifar100", device, 3, 100, [5,10,20],['cifar100']]
config_cifar100_DLA = [24, "DLA", "cifar100", device, 3, 100, [5,10,20],['cifar100']]


config = config_cifar10_densenet
batch = config[0]
model = config[1]
dataset=config[2]
device=config[3]
input_channels=config[4]
num_classes=config[5]
party_nums = config[6]
splits = config[7]

for split in splits:
    partitions = ["homo", "noniid-labeldir", "iid-diff-quantity", get_noniid_label_number_split_name(split)]
    print(split, partitions)
    for partition in partitions:
        model_dir = "%s_%s_%s_%s_b%d" %  (dataset, split, model, partition, batch)
        print(model_dir)
        if not os.path.exists("models/" + model_dir):
            os.mkdir("models/" + model_dir)
        try:
            model_files = os.listdir("models/"+model_dir)
            for party_num in party_nums:
                for i in range(-1, party_num):
                    file_name = "models/%s/party_%d_%d.pkl" % (model_dir, i, party_num)
                    if not os.path.exists(file_name):
                        if i == -1:
                            print("""python3 EMNIST_VGG_and_SpinalVGG_Oracle.py --index %d --partition %s --party_num %d --split %s --device %s --batch %d --dataset %s --model %s --input_channels %d --num_classes %d""" % (i, partition, party_num, split, device, batch, dataset, model, input_channels, num_classes))
#                         print(file_name)
                        else:
                            print("""python3 EMNIST_VGG_and_SpinalVGG.py --index %d --partition %s --party_num %d --split %s --device %s --batch %d --dataset %s --model %s --input_channels %d --num_classes %d""" % (i, partition, party_num, split, device, batch, dataset, model, input_channels, num_classes))
#                         print(file_name)

        except Exception as e:
            print(e)

cifar10 ['homo', 'noniid-labeldir', 'iid-diff-quantity', 'noniid-#label4']
cifar10_cifar10_densenet_homo_b12
python3 EMNIST_VGG_and_SpinalVGG_Oracle.py --index -1 --partition homo --party_num 50 --split cifar10 --device cuda:1 --batch 12 --dataset cifar10 --model densenet --input_channels 3 --num_classes 10
python3 EMNIST_VGG_and_SpinalVGG.py --index 0 --partition homo --party_num 50 --split cifar10 --device cuda:1 --batch 12 --dataset cifar10 --model densenet --input_channels 3 --num_classes 10
python3 EMNIST_VGG_and_SpinalVGG.py --index 1 --partition homo --party_num 50 --split cifar10 --device cuda:1 --batch 12 --dataset cifar10 --model densenet --input_channels 3 --num_classes 10
python3 EMNIST_VGG_and_SpinalVGG.py --index 2 --partition homo --party_num 50 --split cifar10 --device cuda:1 --batch 12 --dataset cifar10 --model densenet --input_channels 3 --num_classes 10
python3 EMNIST_VGG_and_SpinalVGG.py --index 3 --partition homo --party_num 50 --split cifar10 --device cuda:1 --bat

### Check Test Information

In [403]:
config_emnist_SpinalNet = [2, "SpinalNet", "cifar10", device, 1, 62, [100,200,400],["digits","letters","balanced"]]
config_cifar10_densenet = [12, "densenet", "cifar10", device, 3, 10, [50,100,200],['cifar10']]
# config_cifar10_effenetv2_l = [11, "effenetv2_l", "cifar10", device, 3, 10, [50,100,200],['cifar10']]
config_cifar10_SpinalNet = [10, "SpinalNet", "cifar10", device, 3, 10, [50,100,200],['cifar10']]
# config_cifar100_efficientnet_b7 = [22, "effenetv2_l", "cifar100", device, 3, 100, [5,10,20],['cifar100']]
# config_cifar100_SpinalNet = [23, "SpinalNet", "cifar100", device, 3, 100, [5,10,20],['cifar100']]
config_cifar100_DLA = [24, "DLA", "cifar100", device, 3, 100, [5,10,20],['cifar100']]



config = config_cifar100_SpinalNet
batch = config[0]
model = config[1]
dataset=config[2]
device=config[3]
input_channels=config[4]
num_classes=config[5]
party_nums = config[6]
splits = config[7]

conditions = {"parties": {"$size": 1}, "batch":batch, "ensemble":None, "model":model}
# ensemble_selection_results.update_many(conditions,{"$set": { "model": "SpinalNet"}})
test_results = ensemble_selection_results.find(conditions)  # Find all experiment results
data = list(test_results)
length = len(data)
# print(data)
# print(length, length == (101+201+401) * 3 * 4)
num = (np.sum(party_nums.copy()) + 3) * 4 * len(splits)
print(length, num, length == num)

# print(data[1])

for d in data:
    try:
        print(d["meta_data"], d["party_num"], d["ID"], d["local_validation_accuracy"], d["local_test_accuracy"], d["whole_test_accuracy"])
#         ensemble_selection_results.delete_one({"_id":d["_id"]})
    except:
        pass

0 152 False


In [398]:
for i in range(length):
    try:
        batch_number = data[i]["batch"]
    except:
        batch_number = 1
    if batch_number != batch: # only get data with specific batch number
        print(data[i])
    if "local_validation_accuracy" not in data[i].keys() or data[i]["local_validation_accuracy"] == None:
        print(data[i]["parties"], data[i]["meta_data"], data[i]["party_num"], data[i]["_id"])
print(len(data))
tuple_list = []
for i in range(len(data)):
    tuple_list.append((data[i]["meta_data"],data[i]["parties"][0], data[i]["party_num"], data[i]["batch"]))
# tuple_list.append(tuple_list[15])
# tuple_list.append(tuple_list[15])
# tuple_list.append(tuple_list[177])
b = dict(Counter(tuple_list))
duplicates = {key:value for key,value in b.items() if value > 1} #展现重复元素和重复次数
for duplicate in duplicates:
    print(duplicate, duplicates[duplicate])
    ensemble_selection_results.delete_many({"meta_data":duplicate[0], "ID":"["+str(duplicate[1])+"]", 
                                            "party_num":duplicate[2], "batch":duplicate[3]})

0


In [399]:
device = "cuda:5"
for split in splits:
    partitions = ["homo", "noniid-labeldir", "iid-diff-quantity", get_noniid_label_number_split_name(split)]
    for partition in partitions: 
        meta_data_original = "%s_%s_%s_b%d" %  (dataset, split, partition, batch)
        meta_data = "%s_%s_%s_%s_b%d" % (dataset, split, model, partition, batch)
        for party_num in party_nums:
            for i in range(-1, party_num):
                if (meta_data, i, party_num, batch) not in tuple_list and (meta_data_original, i, party_num, batch) not in tuple_list:
#                     print(meta_data, i)
                    if i == -1:
                        print("python3 test_oracle.py --dataset %s --model %s --index %d --partition %s --party_num %d --split %s --batch %d --device %s --input_channels %d --num_classes %d" %
                         (dataset, model, i, partition, party_num, split, batch, device, input_channels, num_classes) )
                    else:
                        print("python3 test.py --dataset %s --model %s --index %d --partition %s --party_num %d --split %s --batch %d --device %s --input_channels %d --num_classes %d" %
                         (dataset, model, i, partition, party_num, split, batch, device, input_channels, num_classes) )

python3 test_oracle.py --dataset cifar10 --model effenetv2_l --index -1 --partition homo --party_num 50 --split cifar10 --batch 10 --device cuda:5 --input_channels 3 --num_classes 10
python3 test.py --dataset cifar10 --model effenetv2_l --index 0 --partition homo --party_num 50 --split cifar10 --batch 10 --device cuda:5 --input_channels 3 --num_classes 10
python3 test.py --dataset cifar10 --model effenetv2_l --index 1 --partition homo --party_num 50 --split cifar10 --batch 10 --device cuda:5 --input_channels 3 --num_classes 10
python3 test.py --dataset cifar10 --model effenetv2_l --index 2 --partition homo --party_num 50 --split cifar10 --batch 10 --device cuda:5 --input_channels 3 --num_classes 10
python3 test.py --dataset cifar10 --model effenetv2_l --index 3 --partition homo --party_num 50 --split cifar10 --batch 10 --device cuda:5 --input_channels 3 --num_classes 10
python3 test.py --dataset cifar10 --model effenetv2_l --index 4 --partition homo --party_num 50 --split cifar10 --bat

## Batch Remove

In [116]:
ensemble_selection_exp_results.delete_many({"method":"last_layer_hierarchical_clustering_selection","batch":8})
ensemble_selection_exp_results.delete_many({"method":"last_layer_kmeans_clustering_selection","batch":8})
ensemble_selection_exp_results.delete_many({"method":"last_layer_spectral_clustering_selection","batch":8})

In [252]:
from dbconfig import mydb
old_data = mydb["old_data"]
old_data.insert_one({"test":(1,2)})

# Generate VBA

In [190]:
table_titles = []

li = [chr(i) for i in range(ord("A"),ord("Z")+1)]
# print(li)
for i in range(26):
    if li[i]>="H":
        for j in range(26):
            table_titles.append(li[i]+li[j])
table_titles = table_titles[7:]
# print(table_titles)
j = 0
for i in range(31,101):
    print('Range("%s4").Select' % table_titles[j])
    j += 1
    print('ActiveCell.FormulaR1C1 = "%dth Best method"' % i)
    print('Range("%s4").Select' % table_titles[j])
    j += 1
    print('ActiveCell.FormulaR1C1 = "%dth Hierarchical Method"' % i)
    print('Range("%s4").Select' % table_titles[j])
    j += 1
    print('ActiveCell.FormulaR1C1 = "%dth Accuracy"' % i)

Range("HH4").Select
ActiveCell.FormulaR1C1 = "31th Best method"
Range("HI4").Select
ActiveCell.FormulaR1C1 = "31th Hierarchical Method"
Range("HJ4").Select
ActiveCell.FormulaR1C1 = "31th Accuracy"
Range("HK4").Select
ActiveCell.FormulaR1C1 = "32th Best method"
Range("HL4").Select
ActiveCell.FormulaR1C1 = "32th Hierarchical Method"
Range("HM4").Select
ActiveCell.FormulaR1C1 = "32th Accuracy"
Range("HN4").Select
ActiveCell.FormulaR1C1 = "33th Best method"
Range("HO4").Select
ActiveCell.FormulaR1C1 = "33th Hierarchical Method"
Range("HP4").Select
ActiveCell.FormulaR1C1 = "33th Accuracy"
Range("HQ4").Select
ActiveCell.FormulaR1C1 = "34th Best method"
Range("HR4").Select
ActiveCell.FormulaR1C1 = "34th Hierarchical Method"
Range("HS4").Select
ActiveCell.FormulaR1C1 = "34th Accuracy"
Range("HT4").Select
ActiveCell.FormulaR1C1 = "35th Best method"
Range("HU4").Select
ActiveCell.FormulaR1C1 = "35th Hierarchical Method"
Range("HV4").Select
ActiveCell.FormulaR1C1 = "35th Accuracy"
Range("HW4").Se

In [189]:
table_titles = []

li = [chr(i) for i in range(ord("A"),ord("Z")+1)]
# print(li)
for i in range(26):
    if li[i]>="F":
        for j in range(26):
            table_titles.append(li[i]+li[j])
table_titles = table_titles[23:]
# print(table_titles)
j = 0
for i in range(21,103):
    print('Range("%s6").Select' % table_titles[j])
    j += 1
    print('ActiveCell.Formula2R1C1 = "=IF(ISNUMBER(RC1),INDEX(R4C2:R4C" & SEMR & ",0,MATCH(LARGE(RC2:RC" & SEMR & ",%d),RC2:RC" & SEMR & ",0)),"""")"'
         % i)
    print('Range("%s6").Select' % table_titles[j])
    j += 1
    print('ActiveCell.Formula2R1C1 = "=IF(ISNUMBER(RC1),INDEX(R5C2:R5C" & SEMR & ",0,MATCH(LARGE(RC2:RC" & SEMR & ",%d),RC2:RC" & SEMR & ",0)),"""")"'
         % i)
    print('Range("%s6").Select' % table_titles[j])
    j += 1
    print('ActiveCell.Formula2R1C1 = "=LARGE(RC2:RC" & SEMR & ",%d)"'
         % i)

Range("FX6").Select
ActiveCell.Formula2R1C1 = "=IF(ISNUMBER(RC1),INDEX(R4C2:R4C" & SEMR & ",0,MATCH(LARGE(RC2:RC" & SEMR & ",21),RC2:RC" & SEMR & ",0)),"""")"
Range("FY6").Select
ActiveCell.Formula2R1C1 = "=IF(ISNUMBER(RC1),INDEX(R5C2:R5C" & SEMR & ",0,MATCH(LARGE(RC2:RC" & SEMR & ",21),RC2:RC" & SEMR & ",0)),"""")"
Range("FZ6").Select
ActiveCell.Formula2R1C1 = "=LARGE(RC2:RC" & SEMR & ",21)"
Range("GA6").Select
ActiveCell.Formula2R1C1 = "=IF(ISNUMBER(RC1),INDEX(R4C2:R4C" & SEMR & ",0,MATCH(LARGE(RC2:RC" & SEMR & ",22),RC2:RC" & SEMR & ",0)),"""")"
Range("GB6").Select
ActiveCell.Formula2R1C1 = "=IF(ISNUMBER(RC1),INDEX(R5C2:R5C" & SEMR & ",0,MATCH(LARGE(RC2:RC" & SEMR & ",22),RC2:RC" & SEMR & ",0)),"""")"
Range("GC6").Select
ActiveCell.Formula2R1C1 = "=LARGE(RC2:RC" & SEMR & ",22)"
Range("GD6").Select
ActiveCell.Formula2R1C1 = "=IF(ISNUMBER(RC1),INDEX(R4C2:R4C" & SEMR & ",0,MATCH(LARGE(RC2:RC" & SEMR & ",23),RC2:RC" & SEMR & ",0)),"""")"
Range("GE6").Select
ActiveCell.Formula2R1C1 = "=I